In [26]:
import os
import time
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
from os import listdir
import pandas as pd
import numpy as np
import glob
import cv2
import json
from os.path import expanduser
import splitfolders
import shutil
from define_path import Def_Path

from tqdm import tqdm

import torch 
import torchvision
from torchvision import models
from torchvision.models.detection.rpn import AnchorGenerator
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn 
import torchvision.transforms as T
from torchvision.transforms import functional as F
from torchsummary import summary
from sklearn.model_selection import train_test_split

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt 
from PIL import Image

import transforms, utils, engine, train
from utils import collate_fn
from engine import train_one_epoch, evaluate

t = torch.cuda.get_device_properties(0).total_memory
print(t)
torch.cuda.empty_cache()

r = torch.cuda.memory_reserved(0)
print(r)
a = torch.cuda.memory_allocated(0)
print(a)
# f = r-a  # free inside reserved

weights_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_sim_b1_e25_v0.pth'

16908615680
1052770304
982210560


In [27]:
# to generalize home directory. User can change their parent path without entering their home directory
path = Def_Path()

parent_path =  path.home + "/Pictures/" + "Data/"

root_dir = parent_path + path.year + "-" + path.month + "-" + path.day + "/"

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch.cuda.set_per_process_memory_fraction(0.9, 0)
print(device)

cuda


In [29]:
def train_transform():
    return A.Compose([
        A.Sequential([
            A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=1), # Random change of brightness & contrast
        ], p=1)
#         A.Resize(640, 480)  # Resize all images to be 640x480
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

In [30]:
def train_test_split(src_dir):
    dst_dir_img = src_dir + "images"
    dst_dir_anno = src_dir + "annotations"
    
    if os.path.exists(dst_dir_img) and os.path.exists(dst_dir_anno):
        print("folders exist")
    else:
        os.mkdir(dst_dir_img)
        os.mkdir(dst_dir_anno)
        
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        shutil.copy(jpgfile, dst_dir_img)

    for jsonfile in glob.iglob(os.path.join(src_dir, "*.json")):
        shutil.copy(jsonfile, dst_dir_anno)
        
    output = parent_path + "split_folder_output" + "-" + path.year + "-" + path.month + "-" + path.day 
    
    splitfolders.ratio(src_dir, # The location of dataset
                   output=output, # The output location
                   seed=42, # The number of seed
                   ratio=(.7, .2, .1), # The ratio of split dataset
                   group_prefix=None, # If your dataset contains more than one file like ".jpg", ".pdf", etc
                   move=False # If you choose to move, turn this into True
                   )
    
    shutil.rmtree(dst_dir_img)
    shutil.rmtree(dst_dir_anno)
    
    return output  
    

In [31]:
class KPDataset(Dataset):
    def __init__(self, root, transform=None, demo=False):                
        self.root = root
        self.transform = transform
        self.demo = demo # Use demo=True if you need transformed and original images (for example, for visualization purposes)
        self.imgs_files = sorted(os.listdir(os.path.join(root, "images")))
        self.annotations_files = sorted(os.listdir(os.path.join(root, "annotations")))
    
    def __getitem__(self, idx):
        img_file = self.imgs_files[idx]
        img_path = os.path.join(self.root, "images", self.imgs_files[idx])
        annotations_path = os.path.join(self.root, "annotations", self.annotations_files[idx])

        img_original = cv2.imread(img_path)
        img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
        
        with open(annotations_path) as f:
            data = json.load(f)
            bboxes_original = data['bboxes']
            keypoints_original = data['keypoints']
            
            # All objects are keypoints on the robot
            bboxes_labels_original = [] 
            bboxes_labels_original.append('base_joint')
            bboxes_labels_original.append('joint2')
            bboxes_labels_original.append('joint3')
            bboxes_labels_original.append('joint4')
            bboxes_labels_original.append('joint5')
            bboxes_labels_original.append('joint6')  

        if self.transform:   
            # Converting keypoints from [x,y,visibility]-format to [x, y]-format + Flattening nested list of keypoints            
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]], where each keypoint is in [x, y]-format            
            # Then we need to convert it to the following list:
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2]
            keypoints_original_flattened = [el[0:2] for kp in keypoints_original for el in kp]
            
            # Apply augmentations
            transformed = self.transform(image=img_original, bboxes=bboxes_original, bboxes_labels=bboxes_labels_original, keypoints=keypoints_original_flattened)
            img = transformed['image']
            bboxes = transformed['bboxes']
            # Unflattening list transformed['keypoints']
            # For example, if we have the following list of keypoints for three objects (each object has two keypoints):
            # [obj1_kp1, obj1_kp2, obj2_kp1, obj2_kp2, obj3_kp1, obj3_kp2], where each keypoint is in [x, y]-format
            # Then we need to convert it to the following list:
            # [[obj1_kp1, obj1_kp2], [obj2_kp1, obj2_kp2], [obj3_kp1, obj3_kp2]]
            keypoints_transformed_unflattened = np.reshape(np.array(transformed['keypoints']), (-1,1,2)).tolist()

            # Converting transformed keypoints from [x, y]-format to [x,y,visibility]-format by appending original visibilities to transformed coordinates of keypoints
            keypoints = []
            for o_idx, obj in enumerate(keypoints_transformed_unflattened):
#                 print("object", obj)
#                 print(" obj index", o_idx)# Iterating over objects
                obj_keypoints = []
                for k_idx, kp in enumerate(obj): # Iterating over keypoints in each object
                    obj_keypoints.append(kp + [keypoints_original[o_idx][k_idx][2]])
                keypoints.append(obj_keypoints)
        
        else:
            img, bboxes, keypoints = img_original, bboxes_original, keypoints_original        
        
        # Convert everything into a torch tensor        
        bboxes = torch.as_tensor(bboxes, dtype=torch.float32)       
        target = {}
        labels = [1, 2, 3, 4, 5, 6]            
        target["boxes"] = bboxes
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are joint positions
        target["image_id"] = torch.tensor([idx])
        target["area"] = (bboxes[:, 3] - bboxes[:, 1]) * (bboxes[:, 2] - bboxes[:, 0])
        target["iscrowd"] = torch.zeros(len(bboxes), dtype=torch.int64)
        target["keypoints"] = torch.as_tensor(keypoints, dtype=torch.float32)
        img = F.to_tensor(img)        
        bboxes_original = torch.as_tensor(bboxes_original, dtype=torch.float32)
        target_original = {}
        target_original["boxes"] = bboxes_original
        target_original["labels"] = torch.as_tensor(labels, dtype=torch.int64) # all objects are glue tubes
        target_original["image_id"] = torch.tensor([idx])
        target_original["area"] = (bboxes_original[:, 3] - bboxes_original[:, 1]) * (bboxes_original[:, 2] - bboxes_original[:, 0])
        target_original["iscrowd"] = torch.zeros(len(bboxes_original), dtype=torch.int64)
        target_original["keypoints"] = torch.as_tensor(keypoints_original, dtype=torch.float32)        
        img_original = F.to_tensor(img_original)

        if self.demo:
            return img, target, img_original, target_original, img_file
        else:
            return img, target, img_file
    
    def __len__(self):
        return len(self.imgs_files)

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as func
import networkx as nx
import torch_geometric.nn as pyg
from torch_geometric.data import Data


class GNNEncoder(pyg.MessagePassing):
    def __init__(self, vertices_dim=5, hidden_dim=128, num_edge_features=4):
        super(GNNEncoder, self).__init__(aggr='add') # 'add' aggregation for summing edge messages
        self.f_enc = nn.Linear(vertices_dim, hidden_dim)
        self.f_e1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.f_v = nn.Linear(hidden_dim, hidden_dim)
        self.f_e2 = nn.Linear(hidden_dim, num_edge_features)

    def forward(self, x, edge_index, edge_features):
        # Step 1: Vertex to Edge (v -> e)
        h1 = self.f_enc(x)
        h_e1 = self.propagate(edge_index, x=h1)
        
        # Step 2: Edge to Vertex (e -> v)
        h2 = self.f_v(h_e1)

        # Step 3: Vertex to Edge (v -> e) again
        h_e2 = self.propagate(edge_index, x=h2)
        h_e2_prob = torch.sigmoid(self.f_e2(h_e2))
        
        return x, h_e2_prob, edge_index, edge_features

    def message(self, x_i, x_j):
        # Concatenating vertex features for edge messages
        return self.f_e1(torch.cat([x_i, x_j], dim=1))

class GNNDecoder(nn.Module):
    def __init__(self, vertices_dim=5, hidden_dim=128, num_vertices=6, num_edge_features=4):
        super(GNNDecoder, self).__init__()
        self.f_e = nn.Linear((vertices_dim * 2)+2, num_edge_features)  # Concatenate two vertices features
        self.f_h = nn.Linear(num_edge_features, vertices_dim)  # Transform h_ij to the same dimension as vertices
        self.f_v = nn.Linear(vertices_dim, vertices_dim)  # Update vertex feature
    
    def forward(self, vertices, h_e2_prob, edges, edge_features):
        edges = edges.T
        print(edges)
        h_source = vertices[edges[:, 0]]
        h_target = vertices[edges[:, 1]]
        h = torch.zeros_like(vertices)
        for idx, (i,j) in enumerate(edges):  # Iterate over edges # changed line
            single_edge_features = edge_features[idx].unsqueeze(0)
            h_ij = h_e2_prob[idx] * self.f_e(torch.cat((h_source[idx].unsqueeze(0), h_target[idx].unsqueeze(0), single_edge_features), dim=1))  # Include edge weights in the input
            h_ij_transformed = self.f_h(h_ij.squeeze())  # Transform h_ij to the same dimension as vertices
            h[j] += h_ij_transformed  # Accumulate edge features to the target vertex

        h_transformed = self.f_v(h.view(-1, vertices.shape[1]))  # Transform h
        h_transformed = h_transformed.view(vertices.shape)  # Reshape back to original shape
        vertices_g = vertices + h_transformed  # Update vertex features

        return vertices_g  # Return vertices_g as the prediction and vertices_g itself as the mean for Gaussian distribution



In [33]:
class OccludedKeyPointLoss(nn.Module):
    def __init__(self, delta=1.0):
        super().__init__()
        self.delta = delta

    def forward(self, vertices_pred, vertices_gt):
        vertices_pred = vertices_pred[:, :3]  # Considering only x, y coordinates, confidence_score
        vertices_gt = vertices_gt[:, :3]  # Considering only x, y coordinates, confidence_score
        # Compute differences
        diff = (vertices_gt - vertices_pred).abs()
        # Compute Huber loss
        huber_loss = torch.where(diff < self.delta, 0.5 * diff**2, self.delta * (diff - 0.5 * self.delta))
        return huber_loss.mean()
    
def visibility_loss (vertices_pred, vertices_gt):    
    return func.cross_entropy(vertices_pred[:, 3], vertices_gt[:, 3])  # Loss based on visibility of keypoints

def edge_loss(edges_prob, edges_gt):
    # Compute the cross-entropy loss
    loss = -torch.sum(edges_gt.to(device) * torch.log(torch.clamp(edges_prob, min=1e-7)))                      
    return loss

def temporal_consistency_loss(y_true_sequence, y_pred_sequence):
    loss = 0
    for t in range(1, len(y_true_sequence)):
        # Selecting the x, y coordinates and visibility for true and predicted sequences
        true_diff = y_true_sequence[t, :, :] - y_true_sequence[t-1, :, :]
        pred_diff = y_pred_sequence[t, :, :] - y_pred_sequence[t-1, :, :]
        loss += torch.mean(torch.abs(true_diff - pred_diff))
    return loss

In [34]:
MISSING_LABEL_TOKEN = -1 

class KeypointPipeline(nn.Module):
    def __init__(self, weights_path, num_vertices):
        super().__init__()

        self.keypoint_model = torch.load(weights_path).to(device)
        self.num_vertices = num_vertices
        self.gnn_encoder = GNNEncoder()
        self.gnn_decoder = GNNDecoder()

    def process_model_output(self, output):
        scores = output[0]['scores'].detach().cpu().numpy()
        high_scores_idxs = np.where(scores > 0.7)[0].tolist()

        post_nms_idxs = torchvision.ops.nms(output[0]['boxes'][high_scores_idxs], 
                                            output[0]['scores'][high_scores_idxs], 0.3).cpu().numpy()

        confidence = output[0]['scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
        labels = output[0]['labels'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
        keypoints = []
        for idx, kps in enumerate(output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()):
            # Setting t_i = 1 because label is found
            keypoints.append(list(map(int, kps[0,0:2])) + [confidence[idx]] + [1] + [labels[idx]])

        # Create a dictionary where the key is the label and the value is the keypoint
        label_to_keypoint = {}
        for keypoint in keypoints:
            label = keypoint[-1]
            if label not in label_to_keypoint or label_to_keypoint[label][-2] < keypoint[-2]:
                label_to_keypoint[label] = keypoint
        sorted_keypoints = [value for key, value in sorted(label_to_keypoint.items())]
        keypoints = torch.stack([torch.tensor(kp) for kp in sorted_keypoints]).float().to(device)
        visibility = keypoints[:, 3].unsqueeze(1)  # Extracting the visibility
        keypoints_visible = keypoints * visibility  # Predicted visible vertices
        edge_index, edge_features = self.get_edge_info(keypoints_visible)
        vertices, self.enc_e, self.edges, self.edge_features = self.gnn_encoder(keypoints_visible, edge_index, edge_features)
        vertices_pred = self.gnn_decoder(vertices, self.enc_e, self.edges, self.edge_features)
        return vertices_pred       
    
    def get_edge_info(self, vertices):
        num_vertices = vertices.shape[0]
        edges = [(i, (i + 1) % num_vertices) for i in range(num_vertices)]
#         edges += [(i, (i - 1) % num_vertices) for i in range(num_vertices)]  # Adding reverse edges if it's undirected
        print("edges while definition", edges)
        edge_features = []
        for edge in edges:
            k1, k2 = vertices[edge[0]][:2], vertices[edge[1]][:2]
            distance = torch.norm(k1 - k2)
            angle = torch.atan2(k2[1] - k1[1], k2[0] - k1[0])
            edge_features.append([distance.item(), angle.item()])

        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous().to(vertices.device)
        edge_features = torch.tensor(edge_features, dtype=torch.float).to(vertices.device)
        return edge_index, edge_features

    def process_image(self, img):
        img = img.unsqueeze(0).to(device)
        # Temporarily set the keypoint model to evaluation mode
        keypoint_model_training = self.keypoint_model.training  # Save the current mode
        self.keypoint_model.eval()
        with torch.no_grad():
            output = self.keypoint_model(img)
        # Set the keypoint model back to its previous mode
        self.keypoint_model.train(keypoint_model_training)
        img = (img[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
        labeled_keypoints = self.process_model_output(output)

        return labeled_keypoints

    def forward(self, imgs):
        outputs = []

        for i in range(imgs.shape[0]):
            labeled_keypoints = self.process_image(imgs[i])
            outputs.append(labeled_keypoints)

        return outputs
    

In [37]:
# Define the model
model = KeypointPipeline(weights_path, num_vertices=6)
model = model.to(device)

# Define the loss
criterion = OccludedKeyPointLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25  # Define your number of epochs
batch_size = 16

KEYPOINTS_FOLDER_TRAIN = train_test_split(root_dir) +"/train" #train_test_split(root_dir) +"/train"
KEYPOINTS_FOLDER_VAL = train_test_split(root_dir) +"/val"
KEYPOINTS_FOLDER_TEST = train_test_split(root_dir) +"/test"

dataset_train = KPDataset(KEYPOINTS_FOLDER_TRAIN, transform=None, demo=False)
dataset_val = KPDataset(KEYPOINTS_FOLDER_VAL, transform=None, demo=False)
dataset_test = KPDataset(KEYPOINTS_FOLDER_TEST, transform=None, demo=False)

data_loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
data_loader_val = DataLoader(dataset_val, batch_size=1, shuffle=False, collate_fn=collate_fn)
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

v = 1

# Initialize sequences for true and predicted keypoints
y_true_sequence = []
y_pred_sequence = []

model.train()
for epoch in range(num_epochs):
    start_time = time.time()
    for i, batch in enumerate(data_loader_train):
        img_tuple, target_dict_tuple, img_files = batch
        print(f"Processing batch {i+1} with images:", img_files)
        
        imgs = [img.to(device) for img in img_tuple]  # Create list of images

        # Process each image individually
        losses = []
        for i in range(len(imgs)):
            img = imgs[i].unsqueeze(0)  # Unsqueeze to add batch dimension

            # Prepare ground truth vertices for the image
            keypoints = target_dict_tuple[i]['keypoints'].to(device)
            print(keypoints.shape)
            # add visibility to ground truth
            visibility = torch.ones((keypoints.shape[0], keypoints.shape[1], 1)).to(device)
            # add labels to ground truth
            labels = torch.arange(1, keypoints.shape[0] + 1, device=keypoints.device).view(-1, 1, 1).float()
            # update gt with vis and labels
            vertices_gt = torch.cat((keypoints, visibility, labels), dim=2).unsqueeze(0)  # Unsqueeze to add batch dimension
            
            vertices_gt = vertices_gt.squeeze()    
            y_true_sequence.append(vertices_gt)

            # Forward pass
            output = model(img)
            vertices_pred = output[0]
            y_pred_sequence.append(vertices_pred)
            
            edges_prob = model.enc_e
            edges = model.edges.T
            edge_features = model.edge_features
            edges_gt = torch.cat((edges, edge_features), dim=1) 

            # Compute loss for the image
            huber_loss = criterion(vertices_pred, vertices_gt)
            ce_loss = edge_loss(edges_prob, edges_gt)
            vis_loss = visibility_loss(vertices_pred, vertices_gt)

            loss = huber_loss + ce_loss + vis_loss
            losses.append(loss)  # Store loss for the image
            
        # Convert true and predicted sequences to tensors
        y_true_tensor = torch.stack(y_true_sequence)
        y_pred_tensor = torch.stack(y_pred_sequence)
        
        # Compute temporal consistency loss
        temporal_loss = temporal_consistency_loss(y_true_tensor, y_pred_tensor)

        # Average loss over all images in the batch
        other_losses = torch.mean(torch.stack(losses))
        
        # Combine temporal loss with other losses
        total_loss = other_losses + temporal_loss

        # Backward pass and optimization
        optimizer.zero_grad()
        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Clear the sequences for the next batch
        y_true_sequence.clear()
        y_pred_sequence.clear()

    end_time = time.time()
    epoch_time = end_time - start_time
    eta = epoch_time * (num_epochs - epoch - 1)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, ETA: {eta} seconds')

model_save_path = f"/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_occ_b{batch_size}_e{num_epochs}_v{v}.pth"

torch.save(model, model_save_path)
    
# Save the state dict of the model, not the entire model
# torch.save(model.state_dict(), model_save_path)
    
torch.save(model, model_save_path)



Copying files: 2662 files [00:00, 19133.23 files/s]
Copying files: 2662 files [00:00, 20043.87 files/s]
Copying files: 2662 files [00:00, 19889.02 files/s]


Processing batch 1 with images: ('001262.rgb.jpg',)
torch.Size([6, 1, 3])
torch.Size([6, 1, 1])
torch.Size([6, 1, 1])
Ground truth keypoints from annotations tensor([[257.9522, 366.9199,   1.0000,   1.0000,   1.0000],
        [257.9597, 283.0131,   1.0000,   1.0000,   2.0000],
        [310.4373, 223.1638,   1.0000,   1.0000,   3.0000],
        [326.0478, 236.8458,   1.0000,   1.0000,   4.0000],
        [395.2607, 307.2739,   1.0000,   1.0000,   5.0000],
        [405.1375, 326.5618,   1.0000,   1.0000,   6.0000]], device='cuda:0')
labeled keypoint {1: [258, 367, 0.9999609, 1, 1], 2: [258, 283, 0.99992776, 1, 2], 4: [326, 237, 0.9998491, 1, 4], 3: [310, 223, 0.9992879, 1, 3], 5: [395, 308, 0.99391985, 1, 5], 6: [403, 327, 0.9924562, 1, 6]}
Keypoints sorted ? tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [310.0000, 223.0000,   0.9993,   1.0000,   3.0000],
        [326.0000, 237.0000,   0.9998,   1.0000,   4.

labeled keypoint {1: [258, 367, 0.99995995, 1, 1], 2: [258, 283, 0.999897, 1, 2], 4: [321, 230, 0.99974877, 1, 4], 3: [304, 218, 0.9995383, 1, 3], 5: [381, 309, 0.99928856, 1, 5], 6: [398, 324, 0.997687, 1, 6]}
Keypoints sorted ? tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [304.0000, 218.0000,   0.9995,   1.0000,   3.0000],
        [321.0000, 230.0000,   0.9997,   1.0000,   4.0000],
        [381.0000, 309.0000,   0.9993,   1.0000,   5.0000],
        [398.0000, 324.0000,   0.9977,   1.0000,   6.0000]], device='cuda:0')
edges while definition [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
tensor([[0, 1],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [5, 0]], device='cuda:0')
[tensor([[ 251.2646,  388.3510, -127.5220,  -15.0564,  -91.5594],
        [ 237.5478,  289.6880, -108.1806,  -13.0988,  -72.2411],
        [ 289.0286,  227.7620,  -98.9329,  -11.2405,  -67.1200],
        [ 

labeled keypoint {1: [258, 367, 0.9999614, 1, 1], 2: [258, 283, 0.999959, 1, 2], 3: [211, 217, 0.9995814, 1, 3], 4: [229, 206, 0.99957997, 1, 4], 6: [195, 93, 0.9975936, 1, 6], 5: [190, 113, 0.99660385, 1, 5]}
Keypoints sorted ? tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   1.0000,   1.0000,   2.0000],
        [211.0000, 217.0000,   0.9996,   1.0000,   3.0000],
        [229.0000, 206.0000,   0.9996,   1.0000,   4.0000],
        [190.0000, 113.0000,   0.9966,   1.0000,   5.0000],
        [195.0000,  93.0000,   0.9976,   1.0000,   6.0000]], device='cuda:0')
edges while definition [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
tensor([[0, 1],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [5, 0]], device='cuda:0')
[tensor([[ 270.3834,  391.3405,  -84.7177,   -7.2145,  -67.3244],
        [ 241.7364,  287.4705, -100.5024,   -9.8615,  -69.1790],
        [ 203.1731,  224.8341,  -82.4856,   -7.9251,  -57.0315],
        [ 2

labeled keypoint {1: [258, 367, 0.9999608, 1, 1], 2: [258, 283, 0.9999192, 1, 2], 4: [190, 235, 0.9997514, 1, 4], 3: [183, 254, 0.99924856, 1, 3], 5: [95, 205, 0.99919194, 1, 5], 6: [92, 186, 0.99543166, 1, 6]}
Keypoints sorted ? tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [183.0000, 254.0000,   0.9992,   1.0000,   3.0000],
        [190.0000, 235.0000,   0.9998,   1.0000,   4.0000],
        [ 95.0000, 205.0000,   0.9992,   1.0000,   5.0000],
        [ 92.0000, 186.0000,   0.9954,   1.0000,   6.0000]], device='cuda:0')
edges while definition [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
tensor([[0, 1],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [5, 0]], device='cuda:0')
[tensor([[257.9232, 376.9376, -76.8666,  -5.1376, -58.6872],
        [245.8052, 285.2272, -92.6168,  -7.0758, -65.5415],
        [178.5607, 260.1012, -77.3511,  -5.7374, -55.0290],
        [177.2183, 234.57

...

labeled keypoint {1: [258, 367, 0.9999609, 1, 1], 2: [258, 283, 0.99992466, 1, 2], 4: [326, 237, 0.99984574, 1, 4], 3: [310, 223, 0.99963117, 1, 3], 5: [421, 263, 0.99918324, 1, 5], 6: [433, 278, 0.99533087, 1, 6]}
Keypoints sorted ? tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [310.0000, 223.0000,   0.9996,   1.0000,   3.0000],
        [326.0000, 237.0000,   0.9998,   1.0000,   4.0000],
        [421.0000, 263.0000,   0.9992,   1.0000,   5.0000],
        [433.0000, 278.0000,   0.9953,   1.0000,   6.0000]], device='cuda:0')
edges while definition [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
tensor([[0, 1],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [5, 0]], device='cuda:0')
[tensor([[ 2.5814e+02,  3.6731e+02,  5.5042e-01,  6.8451e-01, -1.4290e+00],
        [ 2.5808e+02,  2.8331e+02,  8.4162e-01,  7.0072e-01,  3.5568e-01],
        [ 3.1008e+02,  2.2324e+02,  9.3771e-01,  6.

labeled keypoint {1: [258, 367, 0.9999614, 1, 1], 2: [258, 283, 0.9999504, 1, 2], 4: [301, 213, 0.99985516, 1, 4], 3: [282, 207, 0.99971265, 1, 3], 5: [332, 119, 0.9989759, 1, 5], 6: [348, 104, 0.9972875, 1, 6]}
Keypoints sorted ? tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   1.0000,   1.0000,   2.0000],
        [282.0000, 207.0000,   0.9997,   1.0000,   3.0000],
        [301.0000, 213.0000,   0.9999,   1.0000,   4.0000],
        [332.0000, 119.0000,   0.9990,   1.0000,   5.0000],
        [348.0000, 104.0000,   0.9973,   1.0000,   6.0000]], device='cuda:0')
edges while definition [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
tensor([[0, 1],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [5, 0]], device='cuda:0')
[tensor([[2.5808e+02, 3.6712e+02, 9.7249e-01, 6.4080e-01, 5.4428e-02],
        [2.5825e+02, 2.8333e+02, 1.0294e+00, 6.4998e-01, 5.5729e-01],
        [2.8226e+02, 2.0727e+02, 1.1092e+00, 6.4452e-01, 1.9185e

labeled keypoint {1: [258, 367, 0.99996126, 1, 1], 2: [258, 283, 0.999902, 1, 2], 4: [278, 203, 0.999813, 1, 4], 3: [257, 203, 0.9996749, 1, 3], 6: [378, 136, 0.9986098, 1, 6], 5: [365, 154, 0.9969311, 1, 5]}
Keypoints sorted ? tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [257.0000, 203.0000,   0.9997,   1.0000,   3.0000],
        [278.0000, 203.0000,   0.9998,   1.0000,   4.0000],
        [365.0000, 154.0000,   0.9969,   1.0000,   5.0000],
        [378.0000, 136.0000,   0.9986,   1.0000,   6.0000]], device='cuda:0')
edges while definition [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
tensor([[0, 1],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [5, 0]], device='cuda:0')
[tensor([[ 2.5818e+02,  3.6706e+02,  8.4231e-01,  6.7370e-01, -3.0448e-01],
        [ 2.5833e+02,  2.8305e+02,  8.8737e-01,  7.2113e-01,  5.3887e-01],
        [ 2.5734e+02,  2.0305e+02,  9.9424e-01,  6.9798e-

labeled keypoint {1: [258, 367, 0.9999609, 1, 1], 2: [258, 283, 0.99992776, 1, 2], 3: [281, 206, 0.99986064, 1, 3], 4: [301, 213, 0.99984074, 1, 4], 5: [384, 265, 0.9955681, 1, 5], 6: [392, 246, 0.99503434, 1, 6]}
Keypoints sorted ? tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [281.0000, 206.0000,   0.9999,   1.0000,   3.0000],
        [301.0000, 213.0000,   0.9998,   1.0000,   4.0000],
        [384.0000, 265.0000,   0.9956,   1.0000,   5.0000],
        [392.0000, 246.0000,   0.9950,   1.0000,   6.0000]], device='cuda:0')
edges while definition [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
tensor([[0, 1],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [5, 0]], device='cuda:0')
[tensor([[258.2347, 367.3299,   0.7657,   0.5208,  -0.8998],
        [258.1615, 283.3564,   0.9370,   0.4588,   0.5819],
        [281.1871, 206.2850,   1.0393,   0.4966,   1.9432],
        [301.3348, 213

labeled keypoint {1: [258, 367, 0.9999609, 1, 1], 2: [258, 283, 0.99988234, 1, 2], 4: [191, 234, 0.99987507, 1, 4], 3: [183, 254, 0.99981874, 1, 3], 5: [199, 136, 0.99745995, 1, 5], 6: [220, 139, 0.9938392, 1, 6]}
Keypoints sorted ? tensor([[258.0000, 367.0000,   1.0000,   1.0000,   1.0000],
        [258.0000, 283.0000,   0.9999,   1.0000,   2.0000],
        [183.0000, 254.0000,   0.9998,   1.0000,   3.0000],
        [191.0000, 234.0000,   0.9999,   1.0000,   4.0000],
        [199.0000, 136.0000,   0.9975,   1.0000,   5.0000],
        [220.0000, 139.0000,   0.9938,   1.0000,   6.0000]], device='cuda:0')
edges while definition [(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
tensor([[0, 1],
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [5, 0]], device='cuda:0')
[tensor([[257.9579, 366.9860,   1.1214,   0.6345,   0.7648],
        [258.2196, 283.1478,   0.9045,   0.6421,   0.5735],
        [183.3574, 254.1751,   0.9883,   0.6408,   1.6659],
        [191.2929, 234

In [ ]:
def visualize_and_save(img, vertices, filename):
    print("type of image befor conversion",type(img))    
    print("type of vertices before conversion", type(vertices))
    print(img)
    img = (img.permute(1,2,0).cpu().numpy() * 255).astype(np.uint8)
#     img = (img * 255).astype(np.uint8)  # Convert back from [0, 1] range to [0, 255]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    vertices = vertices.cpu().numpy()

    print(f"Image shape before saving: {img.shape}")  # print the image shape
    print("type of vertices", type(vertices))
#     print("entered vertices", vertices)
#     print("entered image", img)

    # Convert grayscale to BGR if necessary
    if len(img.shape) == 2:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        
    for i in range(vertices.shape[0]):
        img = cv2.circle(img, (int(vertices[i, 0]), int(vertices[i, 1])), radius=2, color=(0, 0, 255), thickness=-1)
        
    result = cv2.imwrite(filename, img)
    print(f"Image saved at {filename}: {result}")  # print if save was successful

    # If the image didn't save correctly, save the image data to a text file for examination
    if not result:
        with open(filename + ".txt", "w") as f:
            np.savetxt(f, img.flatten())

In [ ]:
def test_and_save_model(model, data_loader_test):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_trifocal_loss = 0.0
    total_ce_loss = 0.0
    total_vis_loss = 0.0
    num_batches = 0

    y_true_sequence = []
    y_pred_sequence = []

    # We don't need to track gradients during evaluation
    with torch.no_grad():
        for idx, batch in enumerate(data_loader_test):
            img_tuple, target_dict_tuple, img_files = batch

            total_batch_loss = 0.0
            total_batch_trifocal_loss = 0.0
            total_batch_ce_loss = 0.0
            total_batch_vis_loss = 0.0

            # Process each image individually
            for i in range(len(img_tuple)):
                img = img_tuple[i].to(device)
                target = target_dict_tuple[i]

                # Prepare ground truth vertices for the image
                keypoints = target['keypoints'].to(device)
                visibility = torch.ones((keypoints.shape[0], keypoints.shape[1], 1)).to(device)
                vertices_gt = torch.cat((keypoints, visibility), dim=2).unsqueeze(0)  # Unsqueeze to add batch dimension
                vertices_gt = vertices_gt.squeeze()
                y_true_sequence.append(vertices_gt)

                # Forward pass
                output = model(img.unsqueeze(0))
                vertices_pred = output[0]
                y_pred_sequence.append(vertices_pred)

                edges_prob = model.enc_e
                edges = model.edges
                edge_features = model.edge_features
                edges_gt = torch.cat((edges, edge_features), dim=1) 

                trifocal_loss = criterion(vertices_pred, vertices_gt)
                ce_loss = edge_loss(edges_prob, edges_gt)
                vis_loss = visibility_loss(vertices_pred, vertices_gt)
                loss = trifocal_loss + ce_loss + vis_loss

                total_batch_loss += loss.item()
                total_batch_trifocal_loss += trifocal_loss.item()
                total_batch_ce_loss += ce_loss.item()
                total_batch_vis_loss += vis_loss.item()

                # Visualize and save the prediction
                filename = f'/home/jc-merlab/Pictures/Data/occ_vis_data/image_{idx}_{i}.jpg'
                visualize_and_save(img, vertices_pred, filename)
                print(f"Image saved at {filename}")  # Print statement to confirm image save

            # Convert true and predicted sequences to tensors
            y_true_tensor = torch.stack(y_true_sequence)
            y_pred_tensor = torch.stack(y_pred_sequence)

            # Compute temporal consistency loss
            temporal_loss = temporal_consistency_loss(y_true_tensor, y_pred_tensor)

            total_loss += (total_batch_loss + temporal_loss) / len(img_tuple)
            total_trifocal_loss += total_batch_trifocal_loss / len(img_tuple)
            total_ce_loss += total_batch_ce_loss / len(img_tuple)
            num_batches += 1

            # Clear the sequences for the next batch
            y_true_sequence.clear()
            y_pred_sequence.clear()
    
    # Average the loss over all batches
    avg_loss = total_loss / num_batches
    avg_trifocal_loss = total_trifocal_loss / num_batches
    avg_ce_loss = total_ce_loss / num_batches
    
    print(f'Avg. Test Loss: {avg_loss}, Avg. Trifocal Loss: {avg_trifocal_loss}, Avg. Cross Entropy Loss: {avg_ce_loss}')
    return avg_loss, avg_trifocal_loss, avg_ce_loss

In [ ]:
# avg_loss, avg_trifocal_loss, avg_ce_loss, all_preds = test_and_save_model(model, data_loader_test)

avg_loss, avg_trifocal_loss, avg_ce_loss = test_and_save_model(model, data_loader_test)

In [47]:
import cv2
import os

# Directory containing images
dir_path = '/home/jc-merlab/Pictures/Data/occ_vis_data/'
images = []

# Ensure the images are sorted by name
for f in sorted(os.listdir(dir_path)):
    if f.endswith('.jpg') or f.endswith('.png'):  # Check for image file extension
        images.append(f)

# Determine the width and height from the first image
image_path = os.path.join(dir_path, images[0])
frame = cv2.imread(image_path)
cv2.imshow('video',frame)
height, width, channels = frame.shape

# Define the codec and create a VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Be sure to use the correct codec
video_filename = 'output.mp4'
video = cv2.VideoWriter(video_filename, fourcc, 3.0, (width, height))

for image in images:
    image_path = os.path.join(dir_path, image)
    frame = cv2.imread(image_path)
    video.write(frame)  # Write out frame to video

# Release everything when job is finished
video.release()
cv2.destroyAllWindows()

print("The output video is", video_filename)

The output video is output.mp4


In [55]:
model_path = '/home/jc-merlab/Pictures/Data/trained_models/keypointsrcnn_weights_occ_b16_e25_v6.pth'

model = torch.load(model_path).to(device)


image = Image.open("/home/jc-merlab/Pictures/Data/occluded_results_mi20_ma80_n2/occluded_000041.rgb.jpg")
print(type(image))

img = F.to_tensor(image).to(device)
img.unsqueeze_(0)
# print(image.shape)
# image = list(image)
# print(type(images))
# images = list(image.to(device) for image in images)

with torch.no_grad():
    model.to(device)
    model.eval()
    output = model(img)
    
keypoints = output[0]

print(keypoints)
plt.imshow(image)

# Assuming each keypoint is a tensor representing (x, y)
for i, keypoint in enumerate(keypoints):
    print(f'Key point {i}: {keypoint}')
    keypoint = keypoint.cpu().numpy()
    plt.plot(keypoint[0], keypoint[1], 'ro')
plt.show()

# Plotting the image

# plt.imshow(image)

# for keypoint in output[0]:
#     plt.plot(keypoint[0], keypoint[1], 'ro')

# plt.show()

<class 'PIL.JpegImagePlugin.JpegImageFile'>
[[258, 367, 0.9999529, 1, 1], [168, 278, 0.753524, 1, 4], [181, 178, 0.9980096, 1, 5], [203, 174, 0.99722606, 1, 6]]
[(0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 0)]
(0, 1)
0
1
(1, 2)
1
2
(2, 3)
2
3
(3, 4)
3
4


IndexError: index 4 is out of bounds for dimension 0 with size 4